In [2]:
import pandas as pd
from openpyxl import load_workbook

# Путь к файлу
file_path = "aav.xlsx"

# Загружаем книгу (data_only=False, чтобы сохранить формулы)
wb = load_workbook(file_path, data_only=False)

# Функция для чтения листов с формулами
def read_sheet_with_formulas(sheet_name):
    sheet = wb[sheet_name]
    data = []

    for row in sheet.iter_rows():
        row_data = []
        for cell in row:
            # Проверяем, есть ли формула
            if cell.value and isinstance(cell.value, str) and cell.value.startswith("="):
                row_data.append(cell.value)  # Оставляем формулу как текст
            else:
                row_data.append(cell.value)  # Обычные значения
        data.append(row_data)

    return pd.DataFrame(data)

# Читаем все три листа
df_sheet1 = read_sheet_with_formulas("Sheet1")
df_plasmids = read_sheet_with_formulas("Плазмиды")
df_transduction = read_sheet_with_formulas("Трансдукция ААВ")

# Выводим первые строки каждого листа
print("\n=== Sheet1 ===")
print(df_sheet1.head())

print("\n=== Плазмиды ===")
print(df_plasmids.head())

print("\n=== Трансдукция ААВ ===")
print(df_transduction.head())

# Сохраняем в CSV (для удобства)
df_sheet1.to_csv("Sheet1.csv", index=False)
df_plasmids.to_csv("Plasmids.csv", index=False)
df_transduction.to_csv("Transduction_AAV.csv", index=False)



=== Sheet1 ===
                                            0      1             2   \
0                                         None   None          None   
1  перевод единиц объема наработки в объем КЖ    None          None   
2                              Единица объема   Т175   КРУГЛЫЕ 10см   
3                              Объем среды, мл     30            ND   
4                     Площадь поверхности, см2    175           100   

                  3     4     5                             6   \
0               None  None  None                          None   
1               None  None  None                          None   
2  5-level cellstack  None  None               Единица объема    
3                 ND  None  None               Объем среды, мл   
4               3180  None  None  Объем ТФ микса на флакон, мл   

                                        7             8                  9   \
0                                     None          None               None   
1 

In [3]:
import openpyxl

# Открываем Excel файл
wb = openpyxl.load_workbook('aav.xlsx', data_only=False)

# Список названий листов
sheet_names = wb.sheetnames

# Проходим по каждому листу
for sheet_name in sheet_names:
    sheet = wb[sheet_name]
    print(f'Лист: {sheet_name}')
    
    # Проходим по всем ячейкам листа
    for row in sheet.iter_rows():
        for cell in row:
            # Если в ячейке есть формула, выводим её
            if cell.value and isinstance(cell.value, str) and cell.value.startswith('='):
                print(f'Формула в ячейке {cell.coordinate}: {cell.value}')


Лист: Sheet1
Формула в ячейке C6: =C5/B5
Формула в ячейке D6: =D5/B5
Формула в ячейке H6: =H5*B6
Формула в ячейке I6: =H5*C6
Формула в ячейке J6: =D6*H5
Формула в ячейке B9: =IFERROR(VLOOKUP(TRIM(LEFT(A9,FIND("(",A9&"(")-1)),Плазмиды!B:C,2,FALSE),"Не найдено")
Формула в ячейке C9: =B9*660/10^6
Формула в ячейке E9: =C9*D9
Формула в ячейке F9: =SUM(E9:E11)
Формула в ячейке N9: =C9*M10
Формула в ячейке P9: =N9/O9
Формула в ячейке Q9: =SUM(P9:P11)
Формула в ячейке B10: =IFERROR(VLOOKUP(TRIM(LEFT(A10,FIND("(",A10&"(")-1)),Плазмиды!B:C,2,FALSE),"Не найдено")
Формула в ячейке C10: =B10*660/10^6
Формула в ячейке E10: =C10*D10
Формула в ячейке H10: =$B$6
Формула в ячейке J10: =VALUE(LEFT(G9,FIND("x",G9)-1)) * $B$4 * I9*H10
Формула в ячейке L10: =J10*K10
Формула в ячейке M10: =J10/F9
Формула в ячейке N10: =C10*M10
Формула в ячейке P10: =N10/O10
Формула в ячейке R10: =VALUE(LEFT(G9,FIND("x",G9)-1))*H10*$I$9 - Q9*0.001
Формула в ячейке S10: =VALUE(LEFT(G9,FIND("x",G9)-1))*H10*$I$9 - L10*0.001
Форм

In [ ]:
import openpyxl

# Открываем Excel файл
wb = openpyxl.load_workbook('aav.xlsx', data_only=False)

# Список названий листов
sheet_names = wb.sheetnames

# Проходим по каждому листу
for sheet_name in sheet_names:
    sheet = wb[sheet_name]
    print(f'Лист: {sheet_name}')
    
    # Проходим по всем ячейкам листа
    for row in sheet.iter_rows():
        for cell in row:
            # Если в ячейке есть формула, выводим её и координаты (название ячейки)
            if cell.value and isinstance(cell.value, str) and cell.value.startswith('='):
                print(f'Формула в ячейке {cell.coordinate}: {cell.value}')
